In [1]:
# !export PROJECT_ID="your-project-id"

import os
import sys
sys.path.append("../../common/src")
sys.path.append("../src")
PROJECT_ID = os.getenv("PROJECT_ID")

In [2]:
from langchain.agents import Tool, AgentExecutor, BaseMultiActionAgent, AgentOutputParser
from langchain.agents.chat.base import ChatAgent
from langchain.prompts import StringPromptTemplate
from langchain.llms import OpenAI, VertexAI
from langchain.chat_models import ChatVertexAI
from langchain.utilities import SerpAPIWrapper
from langchain.chains import LLMChain
from typing import List, Union
from langchain.schema import AgentAction, AgentFinish, OutputParserException
import re
import config
from config import (VERTEX_LLM_TYPE_BISON_CHAT,
                    LLM_BACKEND_ROBOT_USERNAME,
                    LLM_BACKEND_ROBOT_PASSWORD)
from common.utils.token_handler import UserCredentials

config.RULES_ENGINE_BASE_URL = f"https://{PROJECT_ID}.cloudpssolutions.com/rules-engine/api/v1"
config.auth_client = UserCredentials(LLM_BACKEND_ROBOT_USERNAME,
                              LLM_BACKEND_ROBOT_PASSWORD,
                              f"https://{PROJECT_ID}.cloudpssolutions.com")

INFO: [config/config.py:54 - <module>()] Namespace File not found, setting job namespace as default
INFO: [config/config.py:87 - get_environ_flag()] ENABLE_GOOGLE_LLM = True
INFO: [config/config.py:87 - get_environ_flag()] ENABLE_GOOGLE_MODEL_GARDEN = True
INFO: [config/config.py:87 - get_environ_flag()] ENABLE_LLAMA2CPP_LLM = False
INFO: [config/config.py:87 - get_environ_flag()] ENABLE_OPENAI_LLM = True
INFO: [config/config.py:87 - get_environ_flag()] ENABLE_COHERE_LLM = True
INFO: [config/config.py:87 - get_environ_flag()] ENABLE_TRUSS_LLAMA2 = True
INFO: [config/config.py:248 - <module>()] Can't load llm_service_models.json: 404 Secret [projects/877759600672/secrets/llm_service_password_Llama2cpp] not found or has no versions.
INFO: [config/config.py:260 - <module>()] LLM types loaded ['OpenAI-GPT3.5', 'OpenAI-GPT4', 'Cohere', 'VertexAI-Text', 'VertexAI-Chat-V1', 'VertexAI-Chat']
INFO: [config/config.py:285 - <module>()] Embedding models loaded ['VertexAI-Embedding', 'OpenAI-Embedd

In [3]:
from services.agents.agents import RoutingAgent

dispatch_agent = RoutingAgent(VERTEX_LLM_TYPE_BISON_CHAT)
prefix = dispatch_agent.prefix + """
    An AI Routing Assistant has access to the following routes:
    - [plan] to create and execute a plan.
    - [chat] to perform generic chat conversation.
    - [query_engine_1] to run a query on a query engine for topics about driver license and state id.
    - [query_engine_2] to run a query on a query engine for topics about the latest news in Jordan and the middle east.
    
    Choose one route based on the question below:
    """
dispatch_agent.set_prefix(prefix)
# dispatch_agent.llm_type = "VertexAI-Chat"
# dispatch_agent.agent_type = "langchain_Conversational"
# dispatch_agent.agent_class = StructuredChatAgent

print(dispatch_agent.prefix)

tools = dispatch_agent.get_tools()
agent = dispatch_agent.load_agent()


agent_executor = AgentExecutor.from_agent_and_tools(
      agent=agent, tools=tools, verbose=True)


You are an AI Routing Assistant. An AI Planning Routing
is an AI agent based on a large language model trained by Google.
AI Routing Assistants are designed to be able to evaluate a question from a user
and decide which routes to choose based on the context. The job of an AI Routing
Assistant is to return the best matched agent_name in this format:
'[agent_name] to [perform the action in this route]'.

For example:
- Use [query_engine] to run a query on a query engine with a specific domain knowledge.
- Use [plan] to create and execute a plan.
- Use [chat] to perform generic chat conversation.

AI Routing Assistant will return only ONE route as the format below:
- Use [query_engine] to run a query on a query engine with a specific domain knowledge.

ROUTES:
------

    An AI Routing Assistant has access to the following routes:
    - [plan] to create and execute a plan.
    - [chat] to perform generic chat conversation.
    - [query_engine_1] to run a query on a query engine for topic

In [4]:
from services.agents.agent_service import parse_output

prompt = "How do I apply for a state id?"
agent_inputs = {
    "input": prompt,
    "chat_history": []
  }
output = agent_executor.run(agent_inputs)
routes = parse_output("Route:", output)
print(routes)


ValueError: Missing some input keys: {'\n  "route"'}

In [ ]:
def parse_step(text:str) -> dict:
  step_regex = re.compile(
      r"\d+\.\s.*\[(.*)\]\s?(.*)", re.DOTALL)
  matches = step_regex.findall(text)
  return matches

route, detail = parse_step(routes[0])[0]
route

In [ ]:
prompt = "How do I apply for a driver's license?"
agent_inputs = {
    "input": prompt,
    "chat_history": []
  }
output = agent_executor.run(agent_inputs)
print(output)

In [ ]:
prompt = "What's the latest news in Jordan?"
agent_inputs = {
    "input": prompt,
    "chat_history": []
  }
output = agent_executor.run(agent_inputs)
print(output)

In [ ]:
prompt = "Compose and send an email to all the medicaid applicants that are missing income verification asking them to provide a pay stub from their employers"
agent_inputs = {
    "input": prompt,
    "chat_history": []
  }
output = agent_executor.run(agent_inputs)
print(output)